In [63]:
import numpy as np
import pandas as pd
import requests
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

### 1. Prepare Data
Issuing a GET request and location the first table in the page content

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

req = requests.get(link)
page = BeautifulSoup(req.content).find('table')

### 2. Filtering Data
Extracting just the content from the HTML \< table \> tag. More over, removing borough lines with the value __Not assigned__, besides.
All valid lines has spaces and new line caracters remove from begining and ending string.

In [3]:
data = list()

for trs in page.find_all('tr'):
    line = [td.get_text().strip() for td in trs.find_all('td')]
    if line and line[1] != 'Not assigned':
        data.append(line)

### 3. Create DataFrame
Using data list as a source to create a DataFrame along with all columns name requested. Most importantly, the groupby() function is grouping all registers which have the same Borough and PostalCode values, besides, join all Neighborhood values with a comma.

As the groupby() function generate a Series object, I utilized to_frame() function to convert the result back to DataFrame with a new index created through reset_index() function.

In [4]:
df = pd.DataFrame.from_records(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df.groupby(['PostalCode', 'Borough'], sort=False)['Neighborhood'].apply(', '.join).to_frame().reset_index()

### 4. Removing invalid data
Replacing all __Not assigned__ value from Neighborhood column with the value from Borough column. Finally, display the first 10 rows.

In [5]:
df['Neighborhood'] = df['Borough'].where(df['Neighborhood'] == 'Not assigned', df['Neighborhood'])
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### 5. Size
Dispay the number rows and columns

In [6]:
df.shape

(103, 3)

### 6. Problems found
During this wrinting the geodecode package was not working, so I used the csv available.

### 7. Read Coordinates
Reading a csv file to DataFrame object.

In [7]:
link_coord = 'https://cocl.us/Geospatial_data'
coord = pd.read_csv(link_coord)

### 8. Dataset
Merge both DataFrames, coordinates and neighborhood, based on PostalCode. Then, remove one of the Postal Code column generated by pandas.

In [8]:
df = df.merge(coord, left_on='PostalCode', right_on='Postal Code')
df.drop('Postal Code', axis=1, inplace=True)
df.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### 9. Prepare Geolocation
Define the region which I want to analyse, Toronto. Then, get its geolocation using geolocator object. Finally, use two variables to save longitude and latitude.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### 10. Full Map
Create a map using the folium library and Toronto's geolocation. The zip function is aggregating all necessary information to be used as parameter to the CircleMarker class which will mark each location using a blue circle with radius of 5. Finally, each circle is appended to the map.

In [22]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], 
                                           df['Longitude'], 
                                           df['Borough'], 
                                           df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 11. Filter Neighborhood
Now, a new map will be generated using only boroughs with Toronto in its name.

In [34]:
neighborhood = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
neighborhood.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


 ### 11. Create Map
 Again, all points onto Toronto is a blue circle with radius of 5. The source data is the last dataframe filter during __stage 10__.

In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(neighborhood['Latitude'], 
                                           neighborhood['Longitude'], 
                                           neighborhood['Borough'], 
                                           neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 12. Foursquare settings

In [23]:
CLIENT_ID = 'M3VDRTAU5YXAPWCFKJGUTAGYAGJTCQAD1SP5Y0JZAFPWOUB3' # your Foursquare ID
CLIENT_SECRET = 'OERUVUUZ1EACPOE5TRLI4AOZ5SRYCK10L0MJOULJJFCAHVUX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Function to consume Foursquare API Service. The goal here is return a DataFrame along with Cateogry and venus from places into Tonronto.

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=neighborhood['Neighborhood'],
                                 latitudes=neighborhood['Latitude'], 
                                 longitudes=neighborhood['Longitude'])
toronto_venues

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.654260,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
...,...,...,...,...,...,...,...
1705,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1706,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1707,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1708,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


Group data and count the amount of venue for each Neighborhood

In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",96,96,96,96,96,96
Christie,17,17,17,17,17,17


In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


### 13. Transform Data
Transform Venue Category column to onehot encoder, then inset the Neighborhood column in the new DataFrame

In [48]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

Using the new DataFrame of one hot encoder find the mean.

In [50]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.00,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00


Display the most common venues for each Neighborhhod. The counting is using percentage.

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.03
4  Thai Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1  Seafood Restaurant  0.04
2            Beer Bar  0.04
3                Café  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
               venue  freq
0               Café  0.14
1     Breakfast Spot  0.09
2        Coffee Shop  0.09
3  Convenience Store  0.05
4         Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3                Park  0.06
4          Comic Shop  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport 

This function return the most common venue for each location.

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new DataFrame with the 10 most common venues in each Neighborhood.

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Burger Joint,Cosmetics Shop,Bakery,Steakhouse,Sushi Restaurant,Asian Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Steakhouse,Beer Bar,Seafood Restaurant,Bakery,Café,Cheese Shop,Fish Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Grocery Store,Furniture / Home Store,Convenience Store,Burrito Place,Stadium,Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden Center,Restaurant,Recording Studio,Burrito Place,Skate Park,Auto Workshop,Fast Food Restaurant,Garden
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Plane,Harbor / Marina,Sculpture Garden,Bar,Airport Gate


### 15. Cluster Data
Create K-means clustering model and fit the Toronto's Neighborhood venues

In [61]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Dessert Shop,Shoe Store
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Restaurant,Sporting Goods Shop,Bubble Tea Shop,Pizza Place,Ice Cream Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Bakery,Clothing Store,Italian Restaurant,Breakfast Spot,Cosmetics Shop,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Health Food Store,Other Great Outdoors,Pub,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Farmers Market,Steakhouse,Beer Bar,Seafood Restaurant,Bakery,Café,Cheese Shop,Fish Market


### 16. Final Map


In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 17. Examing Clusters

#### Cluster 1

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Dessert Shop,Shoe Store
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Restaurant,Sporting Goods Shop,Bubble Tea Shop,Pizza Place,Ice Cream Shop
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Bakery,Clothing Store,Italian Restaurant,Breakfast Spot,Cosmetics Shop,Gastropub
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Steakhouse,Beer Bar,Seafood Restaurant,Bakery,Café,Cheese Shop,Fish Market
5,Downtown Toronto,0,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Bubble Tea Shop,Gym / Fitness Center,Bakery,Salad Place
6,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Baby Store,Restaurant,Athletics & Sports,Diner
7,Downtown Toronto,0,Coffee Shop,Café,Bar,Burger Joint,Cosmetics Shop,Bakery,Steakhouse,Sushi Restaurant,Asian Restaurant,American Restaurant
8,West Toronto,0,Pharmacy,Bakery,Supermarket,Middle Eastern Restaurant,Music Venue,Park,Portuguese Restaurant,Café,Brewery,Brazilian Restaurant
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Fried Chicken Joint,Italian Restaurant,Brewery,Pizza Place
10,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Restaurant,Men's Store,French Restaurant,Café,Pizza Place,New American Restaurant,Vietnamese Restaurant


#### Cluster 2

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Garden,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
